In [ ]:
! pip insatll 

In [9]:
"""LLM Chain for generating examples for question answering."""

from __future__ import annotations

from typing import Any

from langchain_core.language_models.base import BaseLanguageModel
from langchain_core.output_parsers import BaseLLMOutputParser
from langchain_core.pydantic_v1 import Field

from langchain.chains.llm import LLMChain
from langchain.evaluation.qa.generate_prompt import PROMPT
from langchain.output_parsers.regex import RegexParser

_QA_OUTPUT_PARSER = RegexParser(
    regex=r"QUESTION: (.*?)\n+ANSWER: (.*)", output_keys=["query", "answer"]
)


class QAGenerateChain(LLMChain):
    """LLM Chain for generating examples for question answering."""

    output_parser: BaseLLMOutputParser = Field(default=_QA_OUTPUT_PARSER)
    output_key: str = "qa_pairs"

    @classmethod
    def is_lc_serializable(cls) -> bool:
        return False

    @classmethod
    def from_llm(cls, llm: BaseLanguageModel, **kwargs: Any) -> QAGenerateChain:
        """Load QA Generate Chain from LLM."""
        return cls(llm=llm, prompt=PROMPT, **kwargs)

ImportError: cannot import name 'field_validator' from 'pydantic' (d:\VS_Code\teacher_student_assist\teacher\Lib\site-packages\pydantic\__init__.cp311-win_amd64.pyd)

In [ ]:
from fastapi import FastAPI, Form, Request, Response, File, Depends, HTTPException, status
from fastapi.responses import RedirectResponse
from fastapi.staticfiles import StaticFiles
from fastapi.templating import Jinja2Templates
from fastapi.encoders import jsonable_encoder
#from langchain.llms import CTransformers
from langchain.chains.qa_generation.base import QAGenerationChain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain.document_loaders import PyPDFLoader
from langchain.prompts import PromptTemplate
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains.summarize import load_summarize_chain
from langchain.chains import RetrievalQA
import os 
import json
import time
import uvicorn
import aiofiles
from PyPDF2 import PdfReader
import csv



ImportError: cannot import name 'field_validator' from 'pydantic' (d:\VS_Code\teacher_student_assist\teacher\Lib\site-packages\pydantic\__init__.cp311-win_amd64.pyd)

In [ ]:
app = FastAPI()

app.mount("/static", StaticFiles(directory="static"), name="static")

templates = Jinja2Templates(directory="templates")



In [ ]:
! pip install ctransformers

In [13]:
from ctransformers import AutoModelForCausalLM


d:\VS_Code\teacher_student_assist\teacher\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
def load_llm():
    # Load the locally downloaded model here
    llm = AutoModelForCausalLM.from_pretrained(
        url="https://rag-llm-api.accubits.cloud/v1/chat/completions",
        model = "meta-llama/Meta-Llama-3-8B-Instruct",
        api_key="budserve_2T9wfwNiiALTt7UAQZCoGuLPAUUTZras9cLUpYZ1",
        model_type="meta-llama",
        max_new_tokens = 500,
        temperature = 0.3
    )
    return llm



In [ ]:
from langchain_community.llms import CTransformers


llm = CTransformers(
    model="meta-llama/Meta-Llama-3-8B-Instruct",
    api_key="budserve_2T9wfwNiiALTt7UAQZCoGuLPAUUTZras9cLUpYZ1",
    model_type="meta-llama",    
    max_new_tokens = 500,    
    temperature = 0.3    )

print(llm.invoke('AI is going to'))

In [31]:
import requests
import json

# Replace with your actual API key
api_key = 'budserve_AorbBLLqy97G7dwGiPH5TDvqvkanMaUCybE8GrHu'

def generate_mcqs(text, num_questions=5):
    url = 'https://rag-llm-api.accubits.cloud/v1/chat/completions'
    headers = {
        'api-key': api_key,
        'Content-Type': 'application/json'
    }
    payload = {
        "model": "meta-llama/Meta-Llama-3-8B-Instruct",
        "max_tokens": 500,
        "stream": True,
        "messages": [
            {
                "role": "system",
                "content": "This is a chat between a user and an artificial intelligence assistant"
            },
            {
                "role": "user",
                "content": f"Generate {num_questions} multiple-choice questions from the following text:\n\n{text}\n\nEach question should have four options and one correct answer."
            }
        ]
    }

    response = requests.post(url, headers=headers, data=json.dumps(payload), stream=True)

    mcqs = ''
    try:
        for chunk in response.iter_lines():
            if chunk:
                chunk_data = chunk.decode('utf-8').replace('data: ', '')
                if chunk_data:
                    try:
                        chunk_json = json.loads(chunk_data)
                        if 'choices' in chunk_json and chunk_json['choices'][0]['delta'].get('content'):
                            mcqs += chunk_json['choices'][0]['delta']['content']
                    except json.JSONDecodeError as e:
                        print(f"Error decoding JSON: {e}")
                        continue
        return mcqs.strip()
    except requests.exceptions.RequestException as e:
        print(f"Request error: {e}")
        return None

# Example usage
text = "The capital of India is New Delhi. The Taj Mahal is located in Agra, Uttar Pradesh."
mcqs = generate_mcqs(text)
if mcqs:
    print(mcqs)

Error decoding JSON: Expecting value: line 1 column 2 (char 1)
Here are 5 multiple-choice questions based on the text:

Question 1: What is the capital of India?
A) Mumbai
B) New Delhi
C) Agra
D) Uttar Pradesh

Correct answer: B) New Delhi

Question 2: Which city is the location of the Taj Mahal?
A) New Delhi
B) Agra
C) Mumbai
D) Uttar Pradesh

Correct answer: B) Agra

Question 3: Which state is the location of Agra?
A) Punjab
B) Uttar Pradesh
C) Maharashtra
D) Gujarat

Correct answer: B) Uttar Pradesh

Question 4: What is the name of the famous monument located in Agra?
A) Red Fort
B) Qutub Minar
C) Taj Mahal
D) Humayun's Tomb

Correct answer: C) Taj Mahal

Question 5: Is Agra located in the state of New Delhi?
A) Yes
B) No
C) Maybe
D) Not mentioned

Correct answer: B) No


In [ ]:
import fitz  # PyMuPDF
import requests
import json

# Replace with your actual API key
api_key = 'budserve_AorbBLLqy97G7dwGiPH5TDvqvkanMaUCybE8GrHu'

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    print(f"Extracted text length: {len(text)}")
    return text

def generate_mcqs(text, num_questions=5):
    url = 'https://rag-llm-api.accubits.cloud/v1/chat/completions'
    headers = {
        'api-key': api_key,
        'Content-Type': 'application/json'
    }
    payload = {
        "model": "meta-llama/Meta-Llama-3-8B-Instruct",
        "max_tokens": 500,
        "stream": True,
        "messages": [
            {
                "role": "system",
                "content": "This is a chat between a user and an artificial intelligence assistant"
            },
            {
                "role": "user",
                "content": f"Generate {num_questions} multiple-choice questions from the following text:\n\n{text}\n\nEach question should have four options and one correct answer."
            }
        ]
    }

    response = requests.post(url, headers=headers, data=json.dumps(payload), stream=True)
    mcqs = ''
    try:
        for chunk in response.iter_lines():
            if chunk:
                chunk_data = chunk.decode('utf-8').replace('data: ', '')
                if chunk_data:
                    try:
                        chunk_json = json.loads(chunk_data)
                        if 'choices' in chunk_json and chunk_json['choices'][0]['delta'].get('content'):
                            mcqs += chunk_json['choices'][0]['delta']['content']
                    except json.JSONDecodeError as e:
                        print(f"Error decoding JSON: {e}")
                        continue
        return mcqs.strip()
    except requests.exceptions.RequestException as e:
        print(f"Request error: {e}")
        return None

def split_text(text, max_length=8000):
    sentences = text.split('. ')
    chunks = []
    current_chunk = ""
    for sentence in sentences:
        if len(current_chunk) + len(sentence) + 1 <= max_length:
            current_chunk += sentence + ". "
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentence + ". "
    if current_chunk:
        chunks.append(current_chunk.strip())
    return chunks

# Example usage
pdf_path = r'D:\VS_Code\teacher_student_assist\students\temp\jesc101.pdf'
pdf_text = extract_text_from_pdf(pdf_path)
text_chunks = split_text(pdf_text)

mcqs = ""
for chunk in text_chunks:
    result = generate_mcqs(chunk)
    if result:
        mcqs += result + "\n\n"

print(mcqs)


In [ ]:


######## pdf mcqs ########

import fitz  # PyMuPDF
import requests
import json
import time

# Replace with your actual API key
api_key = 'budserve_AorbBLLqy97G7dwGiPH5TDvqvkanMaUCybE8GrHu'

def extract_text_from_pdf(pdf_path):
    start_time = time.time()
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    end_time = time.time()
    print(f"Extracted text length: {len(text)}")
    print(f"Time taken to extract text: {end_time - start_time:.2f} seconds")
    return text

def generate_mcqs(text, num_questions=5):
    url = 'https://rag-llm-api.accubits.cloud/v1/chat/completions'
    headers = {
        'api-key': api_key,
        'Content-Type': 'application/json'
    }
    payload = {
        "model": "meta-llama/Meta-Llama-3-8B-Instruct",
        "max_tokens": 500,
        "stream": True,
        "messages": [
            {
                "role": "system",
                "content": "This is a chat between a user and an artificial intelligence assistant"
            },
            {
                "role": "user",
                "content": f"Generate {num_questions} multiple-choice questions from the following text:\n\n{text}\n\nEach question should have four options and one correct answer."
            }
        ]
    }

    start_time = time.time()
    response = requests.post(url, headers=headers, data=json.dumps(payload), stream=True)
    mcqs = ''
    try:
        for chunk in response.iter_lines():
            if chunk:
                chunk_data = chunk.decode('utf-8').replace('data: ', '')
                if chunk_data:
                    try:
                        chunk_json = json.loads(chunk_data)
                        if 'choices' in chunk_json and chunk_json['choices'][0]['delta'].get('content'):
                            mcqs += chunk_json['choices'][0]['delta']['content']
                    except json.JSONDecodeError as e:
                        print(f"Error decoding JSON: {e}")
                        continue
        end_time = time.time()
        print(f"Time taken to generate MCQs: {end_time - start_time:.2f} seconds")
        return mcqs.strip()
    except requests.exceptions.RequestException as e:
        print(f"Request error: {e}")
        return None

def split_text(text, max_length=8000):
    sentences = text.split('. ')
    chunks = []
    current_chunk = ""
    for sentence in sentences:
        if len(current_chunk) + len(sentence) + 1 <= max_length:
            current_chunk += sentence + ". "
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentence + ". "
    if current_chunk:
        chunks.append(current_chunk.strip())
    return chunks

# Example usage
pdf_path = r'D:\VS_Code\teacher_student_assist\students\temp\jesc101.pdf'
pdf_text = extract_text_from_pdf(pdf_path)
text_chunks = split_text(pdf_text)

mcqs = ""
start_time_total = time.time()
for chunk in text_chunks:
    result = generate_mcqs(chunk)
    if result:
        mcqs += result + "\n\n"
end_time_total = time.time()

print(mcqs)
print(f"Total execution time: {end_time_total - start_time_total:.2f} seconds")


Extracted text length: 31769
Time taken to extract text: 0.42 seconds
Error decoding JSON: Expecting value: line 1 column 2 (char 1)
Time taken to generate MCQs: 30.80 seconds
Error decoding JSON: Expecting value: line 1 column 2 (char 1)
Time taken to generate MCQs: 26.36 seconds
Error decoding JSON: Expecting value: line 1 column 2 (char 1)
Time taken to generate MCQs: 25.27 seconds
Error decoding JSON: Expecting value: line 1 column 2 (char 1)
Time taken to generate MCQs: 12.00 seconds
Error decoding JSON: Expecting value: line 1 column 2 (char 1)
Time taken to generate MCQs: 23.35 seconds
Here are five multiple-choice questions based on the provided text:

Question 1: What is the name of the powder formed when magnesium ribbon is burnt in air?

A) Magnesium oxide
B) Zinc oxide
C) Iron oxide
D) Lead oxide

Correct answer: A) Magnesium oxide

Question 2: What is the purpose of balancing a chemical equation?

A) To make the equation_PROD longer
B) To make the equation_RED longer
C) To

In [60]:

########## qdrant mcqs ########

import requests
import json
import os
import time
from qdrant_client import QdrantClient
from qdrant_client.http import models

# Replace with your actual API key and ensure it's securely handled
api_key = 'budserve_AorbBLLqy97G7dwGiPH5TDvqvkanMaUCybE8GrHu'

# Set Qdrant connection details
QDRANT_URL = os.getenv('QDRANT_URL') or "your_qdrant_url_here"  # Replace with actual Qdrant URL if not in env
QDRANT_API_KEY = os.getenv('QDRANT_API_KEY') or "your_qdrant_api_key_here"  # Replace with actual Qdrant API Key

# Connect to Qdrant instance
qdrant_client = QdrantClient(
    url=QDRANT_URL,
    api_key=QDRANT_API_KEY
)
collection_name = "document_embeddings"

def get_chunks_from_qdrant(collection_name):
    start_time = time.time()
    try:
        # Retrieve documents from Qdrant collection
        response = qdrant_client.scroll(collection_name=collection_name, with_payload=True)
        text_chunks = [hit.payload.get('chunk', '') for hit in response[0] if hit.payload.get('chunk', '')]
        
        end_time = time.time()
        print(f"Retrieved {len(text_chunks)} chunks from Qdrant")
        print(f"Time taken to retrieve chunks: {end_time - start_time:.2f} seconds")
        
        if not text_chunks:
            print("No text chunks were found. Please verify the collection contains data with 'text' fields.")
        
        return text_chunks
    except Exception as e:
        print(f"Error retrieving chunks from Qdrant: {e}")
        return []

def generate_mcqs(text_chunk, num_questions=5):
    url = 'https://rag-llm-api.accubits.cloud/v1/chat/completions'
    headers = {
        'api-key': api_key,
        'Content-Type': 'application/json'
    }
    payload = {
        "model": "meta-llama/Meta-Llama-3-8B-Instruct",
        "max_tokens": 500,
        "messages": [
            {
                "role": "system",
                "content": "You are an AI assistant designed to create multiple-choice questions (MCQs) from the provided text. Ensure each MCQ has four options and one correct answer."
            },
            {
                "role": "user",
                "content": f"Here is the text: {text_chunk}. Create {num_questions} multiple-choice questions."
            }
        ]
    }

    print(f"Sending chunk to model: {text_chunk[:500]}...")  # Print first 500 characters for checking
    start_time = time.time()
    response = requests.post(url, headers=headers, data=json.dumps(payload))
    try:
        response_json = response.json()
        mcqs = response_json['choices'][0]['message']['content']
        end_time = time.time()
        print(f"Time taken to generate MCQs: {end_time - start_time:.2f} seconds")
        return mcqs.strip()
    except requests.exceptions.RequestException as e:
        print(f"Request error: {e}")
        return None
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
        print(f"Response text: {response.text}")
        return None
    except KeyError as e:
        print(f"Key error: {e}")
        print(f"Response structure: {response_json}")
        return None

# Example usage
text_chunks = get_chunks_from_qdrant(collection_name)

mcqs = ""
start_time_total = time.time()
for chunk in text_chunks:
    result = generate_mcqs(chunk)
    if result:
        mcqs += result + "\n\n"
end_time_total = time.time()

print(mcqs)
print(f"Total execution time: {end_time_total - start_time_total:.2f} seconds")


Retrieved 5 chunks from Qdrant
Time taken to retrieve chunks: 0.86 seconds
Sending chunk to model: Chemical Reactions and Equations 1 CHAPTER C onsider the following situations of daily life and think what happens when – n milk is left at room temperature during summers. n an iron tawa/pan/nail is left exposed to humid atmosphere. n grapes get fermented. n food is cooked. n food gets digested in our body. n we respire. In all the above situations, the nature and the identity of the initial substance have somewhat changed. We have already learnt about physical and chemical changes of matter in...
Time taken to generate MCQs: 37.51 seconds
Sending chunk to model: four on the RHS. Step III: It is often convenient to start balancing with the compound that contains the maximum number of atoms. It may be a reactant or a product. In that compound, select the element which has the maximum number of atoms. Using these criteria, we select Fe3O4 and the element oxygen in it. There are four oxygen

In [ ]:
! pip install git+https://github.com/BudEcosystem/bud-serve-sdk.git

In [5]:
import openai

In [4]:
from budserve import BudServe

In [22]:
base_url="https://rag-llm-api.accubits.cloud/v1",
model_name="meta-llama/Meta-Llama-3-8B-Instruct",
api_key=os.getenv('llama_api_key')

In [20]:

from budserve.models.langchain import BudServeClient
from langchain_core.prompts import PromptTemplate
from langchain.schema.output_parser import StrOutputParser
import os

api_key=os.getenv('llama_api_key')

llm = BudServeClient(base_url="https://rag-llm-api.accubits.cloud/v1",
                 model_name="meta-llama/Meta-Llama-3-8B-Instruct",
                 api_key=api_key,
                 max_tokens=500)

prompt_template = PromptTemplate.from_template("Tell me a joke about {topic}")

chain = prompt_template | llm | StrOutputParser()

print(chain.invoke("cat"))


Why did the cat join a band?

Because it wanted to be the purr-cussionist!


In [27]:
from budserve import BudServe

client = BudServe(base_url="https://rag-llm-api.accubits.cloud/v1",model_name= model_name,api_key=api_key)

completion = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3-8B-Instruct",
    messages=[{"role": "user", "content": "write an essay about history of cricket"}],
    max_tokens=200
)

print(completion.choices[0].message.content)

Cricket is a sport with a rich and storied history that spans centuries. The origins of the game are shrouded in mystery, with different countries claiming to be the birthplace of the sport. However, it is widely accepted that cricket originated in the 16th century in the English countryside, specifically in the counties of Kent and Sussex.

The earliest known reference to the game is from a 1598 court case in Guildford, Surrey, in which a group of players were accused of playing a "creckett" match on Sundays. The game quickly gained popularity in kapsamında England, with the first official rules being published in 1744. The first cricket club, the Honorable Artillery Company Cricket Club, was established in 1742.

As the British Empire expanded, cricket spread to other parts of the world, including the Indian subcontinent, the West Indies, Australia, and South Africa. The game became a popular pastime among the British colonizers and was often played on


In [2]:
import os
api_key="budserve_AorbBLLqy97G7dwGiPH5TDvqvkanMaUCybE8GrHu"

In [3]:

from llama_index.core import VectorStoreIndex,SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter
from budserve import BudServe

# Load data from the specified directory
documents = SimpleDirectoryReader(r"D:\VS_Code\teacher_student_assist\DATA").load_data()
# Settings.node_parser = SentenceSplitter(chunk_size=512, chunk_overlap=20)

# Initialize the BudServe client
client = BudServe(
    base_url="https://rag-llm-api.accubits.cloud/v1",
    model_name="meta-llama/Meta-Llama-3-8B-Instruct",
    api_key=api_key
)

# Helper function to split and summarize chunks
def summarize_text(text, client, max_tokens=200):
    completion = client.chat.completions.create(
        model="meta-llama/Meta-Llama-3-8B-Instruct",
        messages=[{"role": "user", "content": f"generate the 10 questions and answers for: {text}"}],
        max_tokens=max_tokens
    )
    return completion.choices[0].message.content

# Concatenate the text and split into chunks under token limit
documents_text = "\n\n".join([doc.text for doc in documents])
chunks = [documents_text[i:i+6000] for i in range(0, len(documents_text), 6000)]

# Summarize each chunk and concatenate results
summaries = [summarize_text(chunk, client) for chunk in chunks]
final_summary = "\n\n".join(summaries)

# Print the final summarized text
print(final_summary)


APITimeoutError: Request timed out.